In [65]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

In [66]:
pcd = o3d.io.read_point_cloud("data/output.pcd",remove_nan_points=True, remove_infinite_points=True)

# print(pcd)
# print(np.asarray(pcd.points))
# o3d.visualization.draw_geometries([pcd])

In [71]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd])

In [ ]:
REMOVING OUTLIERS

In [75]:
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=35, std_ratio=3.5)

# 'cl' is the cleaned point cloud
# 'ind' are the indices of inliers
pcd = pcd.select_by_index(ind)

# # Visualize the cleaned point cloud
# o3d.visualization.draw_geometries([pcd])

In [ ]:
segment_models={}
segments={}
max_plane_idx=20
rest=pcd
d_threshold=0.01
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.2,ransac_n=5,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")


geometries = [segments[i] for i in range(max_plane_idx) if i in segments]
o3d.visualization.draw_geometries(geometries)